## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tromso,NO,69.6496,18.9570,17.62,73,0,1.99,clear sky
1,1,Wonthaggi,AU,-38.6044,145.5913,80.94,56,100,5.82,light rain
2,2,Cap Malheureux,MU,-19.9842,57.6142,81.46,70,32,7.00,scattered clouds
3,3,Half Moon Bay,US,37.4636,-122.4286,56.71,91,90,6.91,mist
4,4,Asyut,EG,27.1810,31.1837,56.34,67,97,4.18,overcast clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 70


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
25,25,Cape Town,ZA,-33.9258,18.4232,60.12,91,90,3.00,light intensity drizzle
31,31,East London,ZA,-33.0153,27.9116,67.14,94,75,8.05,light rain
35,35,Parichhatgarh,IN,28.9833,77.9333,69.62,50,53,5.91,broken clouds
43,43,Santa Maria,BR,-29.6842,-53.8069,62.71,92,1,5.10,clear sky
50,50,Pisco,PE,-13.7000,-76.2167,66.25,82,0,12.66,clear sky
60,60,Vila Franca Do Campo,PT,37.7167,-25.4333,61.20,84,31,1.90,scattered clouds
66,66,Hengyang,CN,26.8881,112.6150,63.19,35,0,3.67,clear sky
67,67,Ribeira Grande,PT,38.5167,-28.7000,63.82,71,85,9.95,overcast clouds
73,73,Ponta Do Sol,PT,32.6667,-17.1000,65.03,65,16,9.24,few clouds
79,79,Port Elizabeth,ZA,-33.9180,25.5701,66.49,88,0,9.22,light rain


In [11]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                3
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [17]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna(subset = ["Country"], inplace=True)


C:\Users\Cristhian Noboa\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [18]:
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [19]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
25,Cape Town,ZA,60.12,light intensity drizzle,-33.9258,18.4232,
31,East London,ZA,67.14,light rain,-33.0153,27.9116,
35,Parichhatgarh,IN,69.62,broken clouds,28.9833,77.9333,
43,Santa Maria,BR,62.71,clear sky,-29.6842,-53.8069,
50,Pisco,PE,66.25,clear sky,-13.7000,-76.2167,
60,Vila Franca Do Campo,PT,61.20,scattered clouds,37.7167,-25.4333,
66,Hengyang,CN,63.19,clear sky,26.8881,112.6150,
67,Ribeira Grande,PT,63.82,overcast clouds,38.5167,-28.7000,
73,Ponta Do Sol,PT,65.03,few clouds,32.6667,-17.1000,
79,Port Elizabeth,ZA,66.49,light rain,-33.9180,25.5701,


In [20]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat= row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat}, {lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [21]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
25,Cape Town,ZA,60.12,light intensity drizzle,-33.9258,18.4232,Southern Sun Waterfront Cape Town
31,East London,ZA,67.14,light rain,-33.0153,27.9116,Tu Casa
35,Parichhatgarh,IN,69.62,broken clouds,28.9833,77.9333,Saleem javed home
43,Santa Maria,BR,62.71,clear sky,-29.6842,-53.8069,Hotel Morotin
50,Pisco,PE,66.25,clear sky,-13.7000,-76.2167,Hotel San Isidro Oficial


In [24]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [25]:
# 8a. Create the output File (CSV)
output_data_file= "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [29]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [30]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))